In [1]:
import numpy as np
import pandas as pd

In [2]:
#the datasheets are loaded
playstore=pd.read_csv('googleplaystore.csv')
review=pd.read_csv('googleplaystore_user_reviews.csv')
playstore.head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up


In [3]:
playstore['Reviews'].replace('3.0M','3000000',inplace=True)
playstore['Reviews']=playstore['Reviews'].astype('int')
x=playstore[playstore['Reviews']>5000]
x=x[x['Type']=='Free']
x.shape
x.drop(columns=['Type', 'Price', 'Genres', 'Last Updated'],inplace=True)
x=x[(x['Content Rating']=='Everyone')|(x['Content Rating']=='Teen')]
for i in x['Size']:
    if(i[-1]=='M'):
        x['Size'].replace(i,1024*float(i[:-1]),inplace=True)
    elif(i[-1]=='k'):
        x['Size'].replace(i,float(i[:-1]),inplace=True)
playstore=x
playstore=playstore[playstore['Size']!='Varies with device']
playstore.head()

,App,Category,Rating,Reviews,Size,Installs,Content Rating,Current Ver,Android Ver
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8908.8,"5,000,000+",Everyone,1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25600,"50,000,000+",Teen,Varies with device,4.2 and up
7,Infinite Painter,ART_AND_DESIGN,4.1,36815,29696,"1,000,000+",Everyone,6.1.61.1,4.2 and up
8,Garden Coloring Book,ART_AND_DESIGN,4.4,13791,33792,"1,000,000+",Everyone,2.9.2,3.0 and up
10,Text on Photo - Fonteee,ART_AND_DESIGN,4.4,13880,28672,"1,000,000+",Everyone,1.0.4,4.1 and up


In [4]:
playstore['Size']=playstore['Size'].astype('int')

c:\users\dibyendu das\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [5]:
D={'1,000,000+':1000000, '10,000,000+':10000000, '5,000,000+':5000000, '500,000+':500000, '100,000,000+':100000000, '50,000,000+':50000000, '100,000+':100000, '500,000,000+':500000000,
 '1,000,000,000+':1000000000, '50,000+':50000}
for i in D.keys():
    playstore['Installs'].replace(i,D[i],inplace=True)


c:\users\dibyendu das\appdata\local\programs\python\python37-32\lib\site-packages\pandas\core\generic.py:5890: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [6]:
traffic=playstore.groupby('App')['Size'].mean()*playstore.groupby('App')['Installs'].mean()/(1024**3)

In [7]:
traffic=traffic.reset_index()
traffic.rename(columns={0:'TB Downloaded'},inplace=True)

In [8]:
playstore=playstore.merge(traffic,on='App')

In [9]:
playstore['TB Downloaded']=playstore['TB Downloaded'].astype('int')


In [10]:
z=playstore.groupby('App')['Reviews'].mean()*playstore.groupby('App')['Rating'].mean()/10000
z=z.reset_index()
playstore=playstore.merge(z,on='App')
playstore.rename(columns={0:'Popularity'},inplace=True)

In [11]:
playstore['Popularity']=playstore['Popularity'].astype('int')

In [12]:
current=playstore.groupby('Current Ver')['App'].count()
current=current.reset_index()
playstore=playstore.merge(current,on='Current Ver')
android=playstore.groupby('Android Ver')['App_x'].count()
android=android.reset_index()
playstore=playstore.merge(android,on='Android Ver')
playstore.drop(columns=['Android Ver','Current Ver'],inplace=True)
playstore.rename(columns={'App_x_x':'App','App_y':'Current Ver','App_x_y':'Android Ver'},inplace=True)

In [13]:
category=playstore.groupby('Category')['Rating'].mean()
category=category.reset_index()
playstore=playstore.merge(category,on='Category')

In [14]:
playstore.drop(columns=['Category'],inplace=True)
playstore.rename(columns={'Rating_x':'Rating','Rating_y':'Category'},inplace=True)

In [15]:
playstore.head()

,App,Rating,Reviews,Size,Installs,Content Rating,TB Downloaded,Popularity,Current Ver,Android Ver,Category
0,"U Launcher Lite – FREE Live Cool Themes, Hide ...",4.7,87510,8908,5000000,Everyone,41,41,2,445,4.492857
1,FlipaClip - Cartoon animation,4.3,194216,39936,5000000,Everyone,185,83,2,445,4.492857
2,Colorfit - Drawing & Coloring,4.7,20260,25600,500000,Everyone,11,9,9,445,4.492857
3,Sketch - Draw & Paint,4.5,215644,25600,50000000,Teen,1192,97,48,122,4.492857
4,Infinite Painter,4.1,36815,29696,1000000,Everyone,27,15,1,122,4.492857


In [16]:
playstore.drop(columns=['Installs','Size'],inplace=True)

In [17]:
review=review[review['Sentiment_Polarity']!=0]
review=review[review['Sentiment_Subjectivity']>0.60]
sentiment=review.groupby('App')['Sentiment_Polarity'].mean()*review.groupby('App')['Sentiment_Subjectivity'].mean()

In [18]:
sentiment=sentiment.reset_index()

In [19]:
playstore=playstore.merge(sentiment,on='App')
playstore.rename(columns={0:'Sentiment'},inplace=True)

In [20]:
playstore.corr()['Rating']

Rating           1.000000
Reviews          0.140037
TB Downloaded    0.053128
Popularity       0.143773
Current Ver      0.035016
Android Ver      0.051341
Category         0.355995
Sentiment        0.146337
Name: Rating, dtype: float64

In [21]:
playstore.drop(columns=['App'],inplace=True)

In [22]:
playstore=playstore[['Reviews', 'TB Downloaded','Content Rating','Category','Popularity','Current Ver','Android Ver', 'Sentiment','Rating']]

In [23]:
from sklearn.preprocessing import LabelEncoder

In [24]:
from sklearn.preprocessing import OneHotEncoder

In [25]:
X=playstore.iloc[:,:-1].values
y=playstore.iloc[:,-1].values

In [26]:
encoder=LabelEncoder()
X[:,2]=encoder.fit_transform(X[:,2])

In [27]:
oencoder=OneHotEncoder(categorical_features=[2])
X=oencoder.fit_transform(X).toarray()
X.shape

c:\users\dibyendu das\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\preprocessing\_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
c:\users\dibyendu das\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\preprocessing\_encoders.py:390: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.
  "use the ColumnTransformer instead.", DeprecationWarning)


(538, 9)

In [28]:
X=X[:,1:]
X.shape

(538, 8)

In [29]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

In [30]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=9)

In [31]:
regressor=RandomForestRegressor(n_estimators=300)

In [32]:
regressor.fit(X_train,y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=300, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [33]:
y_pred=regressor.predict(X_test)

In [34]:
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error

In [35]:
r2_score(y_test,y_pred)

0.6369094777317661